In [63]:
import pandas as pd
import numpy as np
import sys


In [121]:


# Load your data (replace 'your_data.csv' with your actual file)
time_period = "14-00--14-15"
date = "20241015"
data_file_name = f"xnas-itch-{date}_{time_period}"
data_stream = pd.read_csv(f'../../../../../data/stocks_nanosecond/{data_file_name}.csv')

print(data_stream.groupby(["sector"]).size())

date_column = 'ts_event'

time_delta = 0
batch_results = []
data_stream[date_column] = pd.to_datetime(data_stream[date_column])
previous_event_timestamp = data_stream[date_column].iloc[0]
previous_check_timestamp = data_stream[date_column].iloc[0]




time_start = pd.Timestamp('2024-10-15 14:00:00.00', tz='UTC')
time_end = pd.Timestamp('2024-10-15 14:00:18.00', tz='UTC')


data_stream[date_column] = pd.to_datetime(data_stream[date_column])
# get data between time_start and time_end
data_stream = data_stream[(data_stream[date_column] >= time_start) & (data_stream[date_column] <= time_end)]




sector
Basic Materials              6424
Communication Services     709795
Consumer Cyclical          901870
Consumer Defensive         324650
Energy                     224675
Financial Services         190653
Healthcare                 259202
Industrials                  9759
Technology                2168461
dtype: int64


In [122]:

print(len(data_stream))

# check ts_event column
print(data_stream[date_column].head())
print(data_stream[date_column].tail())
print(data_stream[date_column].min())
print(data_stream[date_column].max())


125614
0   2024-10-15 14:00:00.000258375+00:00
1   2024-10-15 14:00:00.000260411+00:00
2   2024-10-15 14:00:00.000261986+00:00
3   2024-10-15 14:00:00.000265033+00:00
4   2024-10-15 14:00:00.001097942+00:00
Name: ts_event, dtype: datetime64[ns, UTC]
125609   2024-10-15 14:00:17.997628275+00:00
125610   2024-10-15 14:00:17.997628552+00:00
125611   2024-10-15 14:00:17.998302084+00:00
125612   2024-10-15 14:00:17.998422011+00:00
125613   2024-10-15 14:00:17.998485995+00:00
Name: ts_event, dtype: datetime64[ns, UTC]
2024-10-15 14:00:00.000258375+00:00
2024-10-15 14:00:17.998485995+00:00


In [123]:
print(data_stream.groupby("sector").size())

sector
Basic Materials             473
Communication Services    20580
Consumer Cyclical         19331
Consumer Defensive         7994
Energy                     6701
Financial Services         4471
Healthcare                 6580
Industrials                 255
Technology                59229
dtype: int64


In [124]:
import pandas as pd
import numpy as np


date = "20241015"
data_file_name = f"xnas-itch-{date}_{time_period}"
pre_data_stream = pd.read_csv(f'../../../../../data/stocks_nanosecond/{data_file_name}.csv')

time_start = pd.Timestamp('2024-10-15 14:00:10.00', tz='UTC')
time_end = pd.Timestamp('2024-10-15 14:00:18.00', tz='UTC')


pre_data_stream[date_column] = pd.to_datetime(pre_data_stream[date_column])
# get data between time_start and time_end
pre_data_stream = pre_data_stream[ (pre_data_stream[date_column] <= time_start)]

print("len predatastream: ", len(pre_data_stream))
print(pre_data_stream.groupby("sector").size())




len predatastream:  74010
sector
Basic Materials             289
Communication Services    11028
Consumer Cyclical          8572
Consumer Defensive         4637
Energy                     3325
Financial Services         2822
Healthcare                 4295
Industrials                 115
Technology                38927
dtype: int64


In [ ]:
import secrets
import random


date = "20241015"
data_file_name = f"xnas-itch-{date}_{time_period}"
data_stream = pd.read_csv(f'../../../../../data/stocks_nanosecond/{data_file_name}.csv')



data_stream[date_column] = pd.to_datetime(data_stream[date_column])
# get data between time_start and time_end
data_stream = data_stream[(data_stream[date_column] >= time_start) & (data_stream[date_column] <= time_end)]

print("len of data stream start to end:", len(data_stream))

date_column = 'ts_event'

# # Sort by timestamp (if not already sorted)
# data_stream = data_stream.sort_values(date_column).reset_index(drop=True)
remove_stock_fraction = 0.1
num_excluded = 0
num_total = 0
# Function to downsample technology stocks
def downsample_tech_stocks(row):
    global num_excluded, num_total
    if row['ts_event'] >= time_start and row['sector'] == 'Consumer Cyclical':
        m = secrets.SystemRandom().random()
        print("random number: ", m)
        if m < remove_stock_fraction:  # Randomly exclude some tech stocks
            num_excluded += 1
            num_total += 1
            return None  # Exclude this row
        num_total += 1
    return row
# Apply the downsampling
modified_data = data_stream.apply(downsample_tech_stocks, axis=1).dropna()

print("num_excluded: ", num_excluded, "num_total: ", num_total, "fraction: ", num_excluded/num_total)

print("len of modified_data:", len(modified_data))


result_data =  pd.concat([pre_data_stream, modified_data])
print("len of result_data: ", len(result_data))
# Verify the result
print(result_data.groupby("sector").size())


In [117]:
result_data[date_column].min()

Timestamp('2024-10-15 14:00:00.000258375+0000', tz='UTC')

In [118]:
result_data[date_column].max()

Timestamp('2024-10-15 14:00:17.998485995+0000', tz='UTC')

In [119]:
time_start1 = pd.Timestamp('2024-10-15 14:00:00.00', tz='UTC')

result_data = result_data[(result_data[date_column] >= time_start1)]
print("len of result data: ", len(result_data))
print(result_data.groupby("sector").size())


len of result data:  120309
sector
Basic Materials             473
Communication Services    20580
Consumer Cyclical         14026
Consumer Defensive         7994
Energy                     6701
Financial Services         4471
Healthcare                 6580
Industrials                 255
Technology                59229
dtype: int64


In [120]:
result_data.to_csv(f"random_filtered_data_fraction_{int(remove_stock_fraction*10)}.csv", index=False)